<a href="https://colab.research.google.com/github/sksmslhy/TIL/blob/master/05_%E1%84%87%E1%85%A6%E1%86%A8%E1%84%90%E1%85%A5%E1%84%8B%E1%85%B4_%E1%84%8B%E1%85%B2%E1%84%89%E1%85%A1%E1%84%83%E1%85%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1) 코사인 유사도(Cosine Similarity)**

---
## **1. 코사인 유사도(Cosine Similarity)**
두 벡터 간의 코사인 각도를 이용해 구할 수 있는 두 벡터의 유사도를 의미.  


<img src="https://wikidocs.net/images/page/24603/%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84.PNG">


두 벡터에 A, B에 대해서 코사인 유사도는 식으로 다음과 같이 표현됨  


$similarity=cos(Θ)=\frac{A⋅B}{||A||\ ||B||}=\frac{\sum_{i=1}^{n}{A_{i}×B_{i}}}{\sqrt{\sum_{i=1}^{n}(A_{i})^2}×\sqrt{\sum_{i=1}^{n}(B_{i})^2}}$

문서 단어 행렬에 대해 코사인 유사도를 구해보자.

문서1 : 저는 사과 좋아요  
문서2 : 저는 바나나 좋아요
문서3 : 저는 바나나 좋아요 저는 바나나 좋아요


|-|바나나|사과|저는|좋아요|
|---|---|---|---|---|
|문서1|0|1|1|1|
|문서2|1|0|1|1|
|문서3|2|0|2|2|


In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [ ]:
# 각각에 대한 BoW
doc1=np.array([0,1,1,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,2])

In [ ]:
print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


---
## **2. 유사도를 이용한 추천 시스템 구현하기**
영화 추천 시스템 만들기  
Data Set : [다운로드](https://www.kaggle.com/rounakbanik/the-movies-dataset)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
data = pd.read_csv('movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [ ]:
# 2000개의 데이터만 가지고 학습을 진행할 것임
data = data.head(20000)

In [ ]:
# null값 확인
data['overview'].isnull().sum()

135

In [ ]:
# overview에서 Null 값을 가진 경우에는 Null 값을 제거
data['overview'] = data['overview'].fillna('')

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(20000, 47487)


In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
# 영화 타이틀과 인덱스를 가진 테이블 생성.
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [ ]:
# 영화 타이틀을 입력하면 인덱스를 리턴
idx = indices['Father of the Bride Part II']
print(idx)

4


In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 인덱스 받아옴.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도 구함
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목 리턴
    return data['title'].iloc[movie_indices]

In [ ]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

**코사인 유사도에 관한 재밌는 사이트**
- [wevi](https://ronxin.github.io/wevi/)
- [Word2Vec을 우리 말에](https://word2vec.kr/search/?query=)

---
# **2. 여러가지 유사도 기법**
## **1. 유클리드 거리(Euclidean distance)**
다차원 공간에서 두 개의 점 $p$와 $q$가 각각 $p=(p_{1}, p_{2}, p_{3}, ... , p_{n})$과 $q=(q_{1}, q_{2}, q_{3}, ... , q_{n})$의 좌표를 가질 때 두 점 사이의 거리는 계산하는 유클리드 거리 공식은 다음과 같음.

$\sqrt{(q_{1}-p_{1})^{2}+(q_{2}-p_{2})^{2}+\ ...\ +(q_{n}-p_{n})^{2}}=\sqrt{\sum_{i=1}^{n}(q_{i}-p_{i})^{2}}$

두 점 사이의 거리  
<img src="https://wikidocs.net/images/page/24654/2%EC%B0%A8%EC%9B%90_%ED%8F%89%EB%A9%B4.png">  
직각삼각형으로 표현 가능하므로 피타고라서 정리를 통해 $p$와 $q$ 사이의 거리를 계산할 수 있음.

Example.
다음과 같은 DTM이 있다고 하자.

|-|바나나|사과|저는|좋아요|
|---|---|---|---|---|
|문서1|2|3|0|1|
|문서2|1|2|3|1|
|문서3|2|1|2|2|  


단어의 개수가 4개이므로 4차원 공간에 각각의 문서들을 배치하는 것과 같음. 이들 중 다음 문서Q에 대해 가장 유사한 문서를 찾아내보자.

|-|바나나|사과|저는|좋아요|
|---|---|---|---|---|
|문서Q|1|1|0|1|

# 

In [ ]:
import numpy as np
def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

doc1 = np.array((2,3,0,1))
doc2 = np.array((1,2,3,1))
doc3 = np.array((2,1,2,2))
docQ = np.array((1,1,0,1))

print(dist(doc1,docQ))
print(dist(doc2,docQ))
print(dist(doc3,docQ))

2.23606797749979
3.1622776601683795
2.449489742783178


유클리드 거리의 값이 작을수록 문서 간 거리가 가깝다!

## **2. 자카드 유사도(Jaccard similarity)**
idea : 합집합에서 교집합의 비율을 구하자. (두 문서에서 공통적으로 등장한 단어의 비율~)  
자카드 유사도는 0과 1 사이의 값을 가지며 자카드 유사도를 구하는 함수를 J라고 했을 때, 자카드 유사도 함수 J는 다음과 같다.  

$J(A,B)=\frac{|A∩B|}{|A∪B|}=\frac{|A∩B|}{|A|+|B|-|A∩B|}$  


$J(doc_{1},doc_{2})=\frac{doc_{1}∩doc_{2}}{doc_{1}∪doc_{2}}$

In [ ]:
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

# 토큰화 결과 출력
print(tokenized_doc1)
print(tokenized_doc2)

['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [ ]:
# 두 문서의 합집합
union = set(tokenized_doc1).union(set(tokenized_doc2))
print(union)

{'holder', 'likey', 'coupon', 'you', 'card', 'love', 'banana', 'watch', 'everyone', 'passport', 'apple', 'like'}


In [ ]:
# 두 문서의 교집합
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print(intersection)

{'apple', 'banana'}


In [ ]:
# 교집합의 수를 합집합의 수로 나누면 자카드 유사도가 된다.
print("문서1과 문서2의 Jaccard similarity :", len(intersection)/len(union))

문서1과 문서2의 Jaccard similarity : 0.16666666666666666
